# Visualisation et détection de communautés d'auteurs 

## Importation des modules

In [1]:
# Modules de base
import numpy as np
import pandas as pd
import sys
# Ajout d'un chemin
sys.path.append('..')

# Module de graphes
import networkx as nx
from networkx.algorithms.community import girvan_newman
from sknetwork.clustering import Louvain, Leiden, PropagationClustering

# Modules de représentation graphique
import matplotlib.pyplot as plt

## Importation des données

### Importation des noms des auteurs

In [2]:
# Importation des données
data_authors = pd.read_csv('../data/authors.csv')
data_authors.head()

,id,name
0,1,Hisashi Suzuki
1,10,David Brady
2,100,Santosh S. Venkatesh
3,1000,Charles Fefferman
4,10000,Artur Speiser


### Importation des données papiers-auteurs

In [4]:
# Importation des données
data_paper_authors = pd.read_csv('../data/paper_authors.csv')
# Sélection des 1000 premiers éléments
#data_paper_authors.iloc[:50]
data_paper_authors.head()

,id,paper_id,author_id
0,1,63,94
1,2,80,124
2,3,80,125
3,4,80,126
4,5,80,127


## Construction du graphe des auteurs

### Construction

In [5]:
# Création du jeu de données des arêtes en appariant les auetrus sur les papiers qu'ils ont écrit
data_edges = pd.merge(left=data_paper_authors.rename({'author_id' : 'source'}, axis=1), right=data_paper_authors.rename({'author_id' : 'target'}, axis=1), on='paper_id', how='outer').drop(['id_x', 'id_y'], axis=1)
# Le graphe n'étant pas orienté, on ne conserve d'une moitié les liens
data_edges = data_edges.loc[data_edges['source'] < data_edges['target']]
# Création du graphe à partir des arêtes
G = nx.from_pandas_edgelist(data_edges, 'source', 'target')
data_edges.head()

,paper_id,source,target
2,80,124,125
3,80,124,126
4,80,124,127
7,80,125,126
8,80,125,127


## Détection des communautés d'auteurs

In [5]:
# Detect communities using the Girvan-Newman algorithm
communities = girvan_newman(G)
top_level_communities = next(communities)

KeyboardInterrupt: 

In [ ]:
# CConversion du graphe en une matrice d'adjacences
adjacency = nx.to_numpy_array(G=G)

# Utilisation de l'algorithme de Louvain pour détecter les communautés
louvain = Louvain()
labels = louvain.fit_transform(adjacency)

## Identification d'auteurs remarquables

In [8]:
# Calcul de diverses mesures de centralité
# Le degré de centralité identifie les noeuds avec le plus grand nombre de connexions
degree_centrality = nx.degree_centrality(G)
# La "Betweenness Centrality" identifie les noeuds qui se comportent comme des ponts entre différentes parties de graphe
betweenness_centrality = nx.betweenness_centrality(G)
# La "Closeness Centrality" identifie les noeufs qui sont les plus proches de l'ensemble des autres noeuds
closeness_centrality = nx.closeness_centrality(G)
# La "Eigenvector Centrality" identifie les noeufs qui sont connectés à d'autres noeuds très connectés
eigenvector_centrality = nx.eigenvector_centrality(G)

# Fonction retournant les 5 auteurs les plus centraux pour la notuion de centralité choisie
def get_top_authors(centrality, top_n=5):
    return sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:top_n]

# Calcul des auteurs les plus centraux
top_degree = get_top_authors(degree_centrality)
top_betweenness = get_top_authors(betweenness_centrality)
top_closeness = get_top_authors(closeness_centrality)
top_eigenvector = get_top_authors(eigenvector_centrality)

# Affichage des auteurs
print("Top Authors by Degree Centrality:", top_degree)
print("Top Authors by Betweenness Centrality:", top_betweenness)
print("Top Authors by Closeness Centrality:", top_closeness)
print("Top Authors by Eigenvector Centrality:", top_eigenvector)

Top Authors by Degree Centrality: [(1472, 0.012104768861525618), (178, 0.010643848481686318), (330, 0.00918292810184702), (3001, 0.00845246791192737), (1853, 0.007930710633413336)]
Top Authors by Betweenness Centrality: [(330, 0.06416619215350029), (1472, 0.057548981712559774), (1020, 0.04755189021982697), (2816, 0.031312000527458445), (178, 0.02999208670395666)]
Top Authors by Closeness Centrality: [(330, 0.20474856851675816), (1020, 0.20080109307015004), (1472, 0.19891131444328328), (121, 0.19548122808451948), (1410, 0.19198840119021632)]
Top Authors by Eigenvector Centrality: [(5523, 0.2912695305092457), (7405, 0.2471204696341376), (4280, 0.23706169411424516), (5571, 0.23216939801315495), (9030, 0.21842034827828796)]
